# Numerical Data - Cleanup

<br>
**Author** Mary Letey <br>
**Date** 4/16/18 <br>
**File** megaset.ipynb <br>
**Purpose** Creating a mega-dataset aggregating all previous data sets, sorted by date for the last 5 years

<br><br>

In [2]:
# Imports
import numpy as np
import pandas as pd
import datetime
from dateutil import parser
import matplotlib.pylab as plt
%matplotlib inline

In [3]:
def cleanup(df,name):
    dates = list(df["date"]);
    newdates = []
    for i in range(0,len(dates)):
        mystr = dates[i]
        newdates.append(parser.parse(mystr))
    df["date"] = newdates
    df.sort_values(df.columns[1])
    title = name+"-title"
    body = name+"-body"
    
    titles = list(df["title"])
    bodies = list(df["body"])
    
    df = pd.DataFrame({'date': newdates, title: titles, body: bodies})
    columnsTitles=["date",title,body]
    df = df.reindex(columns = columnsTitles)
    return df

In [4]:
def combine(df1,df2,name):
    
    title = name+"-title"
    body = name+"-body"
    
    date1 = list(df1["date"])
    title1 = list(df1[title])
    body1 = list(df1[body])
    date2 = list(df2["date"])
    title2 = list(df2[title])
    body2 = list(df2[body])
    
    newdate = date1 + date2
    newtitle = title1 + title2
    newbody = body1 + body2
    
    df = pd.DataFrame({'date': newdate, title: newtitle, body: newbody})
    
    return df

In [6]:
def cleanup_wsj(df, index):
    df = df.drop(index)
    new_dates = []
    for date_str in df['date']:
        new_date_str = date_str.replace('Updated ', '')
        new_date_str = new_date_str.replace(' ET', '')
        new_date_str = new_date_str.replace(' | WSJ Pro', '')
        new_dates.append(parser.parse(new_date_str))
#         new_dates.append(new_date_str)
    date_df = pd.DataFrame({'date' : new_dates})
    df['date'] = date_df['date']
    
    return df

In [7]:
dell_wsj_df = pd.read_csv('../data/nlp/dell_5yrs_wsj.csv')
dell_wsj_df = cleanup_wsj(dell_wsj_df, [15,16])
columnsTitles=["date","title","body"]
dell_wsj_df = dell_wsj_df.reindex(columns = columnsTitles)
title = "dell"+"-title"
body = "dell"+"-body"
dates = list(dell_wsj_df["date"])
titles = list(dell_wsj_df["title"])
bodies = list(dell_wsj_df["body"])
dell_wsj_df = pd.DataFrame({'date': dates, title: titles, body: bodies})
columnsTitles=["date",title,body]
dell_wsj_df = dell_wsj_df.reindex(columns = columnsTitles)
dell_wsj_df.head()

,date,dell-title,dell-body
0,2013-02-06 05:44:00,Lenovo’s Diplomatic Response to Dell Buyout,BLOOMBERGDell Inc.’s $24.4 billion deal to tak...
1,2013-04-22 12:50:00,Blackstone Ends Pursuit of Dell,Blackstone Group LP has ended its pursuit of D...
2,2013-02-05 20:50:00,Dell to Sell Itself for $24.4 Billion,Dell Inc. on Tuesday struck a $24.4 billion de...
3,2013-04-23 22:00:00,Dell Approves $91.1 Million in Retention Bonuses,Dell Inc. disclosed Tuesday that its board app...
4,2013-05-06 18:57:00,Dell Deal Negotiator Walks Fine Line,"Last August, Alex Mandl was hosting dinner gue..."


In [157]:
# -- dell 
df = pd.read_csv("../data/nlp/dell_5yrs_seekingalpha.csv");
dfdell = cleanup(df,"dell")
dfdell = combine(dfdell, dell_wsj_df,"dell")
dfdell.sort_values(dfdell.columns[1])

# -- hp 
df = pd.read_csv("../data/nlp/hp_5yrs_fool.csv");
dfhp = cleanup(df,"hp")
df = pd.read_csv("../data/nlp/hp_5yrs_seekingalpha.csv")
df = cleanup(df,"hp")
dfhp = combine(dfhp,df,"hp")

# -- ibm 
df = pd.read_csv("../data/nlp/ibm_5yrs_fool.csv");
dfibm = cleanup(df,"ibm")
df = pd.read_csv("../data/nlp/ibm_5yrs_seekingalpha.csv")
df = cleanup(df,"ibm")
dfibm = combine(dfibm,df,"ibm")

# -- seagate
df = pd.read_csv("../data/nlp/seagate_5yrs_fool.csv");
dfseagate = cleanup(df,"seagate")
df = pd.read_csv("../data/nlp/seagate_5yrs_seekingalpha.csv")
df = cleanup(df,"seagate")
dfseagate = combine(dfseagate,df,"seagate")

# -- western_digital
df = pd.read_csv("../data/nlp/western_digital_5yrs_fool.csv");
dfwestern_digital = cleanup(df,"western-digital")
dfwestern_digital.sort_values(dfwestern_digital.columns[0])

#total_df = pd.merge(dfdell, dfhp, on="date")
#total_df = pd.merge(total_df, dfibm, on="date")
#total_df = pd.merge(total_df, dfseagate, on="date")
total_df = dfdell.merge(dfwestern_digital, on="date", how="outer")
total_df = total_df.merge(dfhp, on="date", how="outer")
total_df = total_df.merge(dfibm, on="date", how="outer")
total_df = total_df.merge(dfseagate, on="date", how="outer")

df_otherdata = pd.read_csv('../data/data-cleanup/combined_data_set.csv')
dates = df_otherdata["date"]
newdates = []
for i in range(0,len(dates)):
    mystr = dates[i]
    newdates.append(parser.parse(mystr))
df_otherdata["date"] = newdates

#test = df_otherdata["dell_Last Price"]
#type(test[1])

total_df = total_df.merge(df_otherdata, on="date", how="outer")

In [156]:
df_otherdata.head()

,Unnamed: 0,date,dell_gtrends,hp_gtrends,ibm_gtrends,seagate_gtrends,westdig_gtrends,dell_Last Price,dell_Volume,hp_Last Price,...,seagate_Last Price,seagate_Volume,westdig_Last Price,westdig_Volume,dow_Last Price,dow_Volume,nasdaq_Last Price,nasdaq_Volume,sandp_Last Price,sandp_Volume
0,0,2018-03-27,36.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2018-03-26,18.0,33.0,NaN,0.0,23.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2018-03-25,0.0,37.0,0.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2018-03-24,21.0,0.0,5.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2018-03-23,55.0,17.0,5.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [167]:
dfwestern_digital.head()

,date,western-digital-title,western-digital-body
0,2013-02-27 16:30:00,What Is Netflix Really Worth?,"Hi, my name is Anders. I'm a Netflix (NASDAQ:N..."
1,2013-11-19 14:30:00,Can Marvell Technology Beat Out Broadcom and LSI?,Marvell Technology Group (NASDAQ:MRVL) will re...
2,2013-12-20 11:05:00,Nike Delivers Solid Results And The Market Yawns,So much attention is given to the growth traje...
3,2013-09-11 15:08:00,Yum! Brands: A Global Growth Story,Investors have lost focus on just how profit...
4,2013-10-26 09:05:00,"What If Lululemon, Under Armour, and Nike Are ...",Under Armour (NYSE:UAA) reported earnings Thur...


In [135]:
dfdell.head()

,date,dell-body,dell-title
0,2013-03-18 00:00:00,(At Least Not At These Prices)Despite differen...,"Dude, You're Not Getting DELL...Or AUTO"
1,2013-03-14 00:00:00,The media is currently focused on the saga of ...,Critical Analysis Of Dell Buyout Plan Shows Hi...
2,2013-03-14 00:00:00,Let me go through a EBIT multiple valuation me...,The Value Of Dell Using The EBIT Multiple Valu...
3,2018-02-07 00:00:00,Henry Blodget is pulling on exactly the right ...,Dell LBO Deal Structure - Like Buying A Rental...
4,2013-03-12 08:55:00,"In the view of many investors, billionaire Car...",Billionaire Carl Icahn Moving Into Multilevel-...


In [160]:
date = list(total_df["date"])
print(len(date))

6847


In [165]:
total_df.head(10)

,date,dell-body,dell-title,western-digital-title,western-digital-body,hp-body,hp-title,ibm-body,ibm-title,seagate-body,...,seagate_Last Price,seagate_Volume,westdig_Last Price,westdig_Volume,dow_Last Price,dow_Volume,nasdaq_Last Price,nasdaq_Volume,sandp_Last Price,sandp_Volume
0,2013-03-18 00:00:00,(At Least Not At These Prices)Despite differen...,"Dude, You're Not Getting DELL...Or AUTO",NaN,NaN,(At Least Not At These Prices)Despite differen...,"Dude, You're Not Getting DELL...Or AUTO",NaN,NaN,NaN,...,35.29,10275000.0,49.600,4016000.0,14452.06,182492000.0,3237.590,492288000.0,1552.10,626351000.0
1,2013-03-14 00:00:00,The media is currently focused on the saga of ...,Critical Analysis Of Dell Buyout Plan Shows Hi...,NaN,NaN,Fonts and typefaces fall into that group of te...,Monotype Imaging Looks To Print Money,NaN,NaN,NaN,...,35.89,9957000.0,50.715,4272000.0,14539.14,89457000.0,3258.929,440563000.0,1563.23,582264000.0
2,2013-03-14 00:00:00,Let me go through a EBIT multiple valuation me...,The Value Of Dell Using The EBIT Multiple Valu...,NaN,NaN,Fonts and typefaces fall into that group of te...,Monotype Imaging Looks To Print Money,NaN,NaN,NaN,...,35.89,9957000.0,50.715,4272000.0,14539.14,89457000.0,3258.929,440563000.0,1563.23,582264000.0
3,2018-02-07 00:00:00,Henry Blodget is pulling on exactly the right ...,Dell LBO Deal Structure - Like Buying A Rental...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,49.55,8941000.0,83.710,4731000.0,24893.35,110920000.0,7051.983,506262000.0,2681.66,600918000.0
4,2013-03-12 08:55:00,"In the view of many investors, billionaire Car...",Billionaire Carl Icahn Moving Into Multilevel-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2013-01-25 00:00:00,With the buyout talks of Dell heating up and t...,Seagate Technology: This Buyout Bust Still Mad...,NaN,NaN,NaN,NaN,IntroductionCognizant Technology Solutions (NA...,Cognizant - Why This Technology Multi-Bagger I...,With the buyout talks of Dell heating up and t...,...,37.25,5962000.0,48.080,2444000.0,13895.98,95989000.0,3149.712,282163000.0,1502.96,399877000.0
6,2013-02-16 00:00:00,When you combine the most oft discussed invest...,5 Thoughts On The Berkshire/3G - Heinz Deal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2013-01-08 18:13:00,The PC industry is affected by both cyclical t...,The Cyclical And Secular Risks Of The PC Industry,NaN,NaN,The PC industry is affected by both cyclical t...,The Cyclical And Secular Risks Of The PC Industry,NaN,NaN,The PC industry is affected by both cyclical t...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2013-01-18 00:00:00,My Seeking Alpha fellow commentator Paulo Sant...,The Best Way To Invest In Technology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,35.38,4976000.0,44.810,3021000.0,13649.70,135210000.0,3134.705,519938000.0,1485.98,754008000.0
9,2013-01-22 00:00:00,After looking into the valuation of Dell (NASD...,"Microsoft: 3.1% Dividend Yield, Good Upside, L...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.89,7387000.0,47.360,2216000.0,13712.21,163271000.0,3143.177,399671000.0,1492.56,640867000.0


In [164]:
pd.DataFrame.to_csv(total_df, "../TOTAL-dataset.csv")